In [ ]:
  # Newton Raphson method 1

import sys, numpy as np, matplotlib.pyplot as plt

s = 0.6 # the bargaining power of the workers
psi_e = 0
psi_u = 0.5
psi_a = 1
n = 1
y_u = 0.4
y_a = 0.4
bar_p = 1
y = 1
c = 0.1
delta = 0.1

# let the matching function in labor market
# m(u,v)=kappa*u**zeta*v**(1-zeta)
# then q(theta) = m(u,v)/v = kappa*theta**(-zeta),f(theta) = m(u,v)/u = kappa*theta**(1-zeta)

kappa = 0.5
zeta = 0.35

'''
q = kappa* (theta) ** (-zeta)
f = theta*q
b = (n-u)*(1+psi_e)+u*(1+psi_u)+a*(1+psi_a)
w = (1-s)*y_u + s*pi

'''

# x = (u, theta, p)
# profit = m(u,v)y*(p-c)-m(u,v)*w

def func1(x):
# use np.sign(a)*(np.abs(a))**(-zeta) to avoid Numpy not allowing fractional powers of negative numbers

    return c-kappa*np.sign(x[1])*(np.abs(x[1]))**(-zeta)*(1-s)*(y*(x[2]-c)-y_u)

def func2(x):
    b = (n-x[0])*(1+psi_e)+x[0]*(1+psi_u)+a*(1+psi_a)
    w = (1-s)*y_u + s*y*(x[2]-c)
    m = kappa*x[0]**zeta*(x[0]*x[1])**(1-zeta)

    return (n-x[0])*(1+psi_e)/b*(1-2*psi_e/(1+psi_e)*(n-x[0]-a)/b)*w/x[2]*(x[2]-c)\
+x[0]*(1+psi_u)/b*(1-2*psi_u/(1+psi_u)*x[0]/b)*y_u/x[2]*(x[2]-c)\
+a*(1+psi_a)/b*(1-2*psi_a/(1+psi_a)*a/b)*y_a/x[2]*(x[2]-c)-m*y*(x[2]-c)+m*w

def func3(x):
    return x[0]*kappa*np.sign(x[1])*(np.abs(x[1]))**(1-zeta)-(n-x[0])*delta

func = [func1, func2, func3]

from autograd import grad, jacobian

a = 0.5

x = np.array([0.01,1,0.1], dtype=float)
x_0 = x.copy()

grad_func0 = grad(func1)
grad_func1 = grad(func2)
grad_func2 = grad(func3)

jacobi = [grad_func0(x), grad_func1(x), grad_func2(x)]

x_1 = x_0 - np.matmul(np.linalg.inv(jacobi),[func1(x_0), func2(x_0), func3(x_0)])
error = np.linalg.norm(x_1-x_0)

iteration = 0
maxiteration = 10**2
threshold = 10**(-2)
t_0 = np.array([0.1, 0.5, 1], dtype=float)

while error > threshold and (iteration < maxiteration):
    iteration += 1
    t_1 = t_0 - np.matmul(np.linalg.inv(jacobi),[func1(t_0), func2(t_0), func3(t_0)])
    error = np.linalg.norm(t_1-t_0)
    t_0 = t_1
    print("x[", iteration, ']=', t_0 )

x[ 1 ]= [0.19491469 0.86528246 0.81158014]
x[ 2 ]= [0.16858801 3.20240073 0.65661713]
x[ 3 ]= [-0.01263441  6.97291132  0.52453244]
x[ 4 ]= [nan nan nan]


<ipython-input-1-41c5a9716f78>:43: RuntimeWarning: invalid value encountered in double_scalars
  m = kappa*x[0]**zeta*(x[0]*x[1])**(1-zeta)


In [ ]:
# Newton Raphson method 1
# solving P236 ex. 16 in the book written by Qingyang Li

import numpy as np, matplotlib.pyplot as plt
from autograd import grad, jacobian

def func1(x):
# use np.sign(a)*(np.abs(a))**(-zeta) to avoid Numpy not allowing fractional powers of negative numbers

    return (x[0])**2-10*x[0]+(x[1])**2+8

def func2(x):

    return x[0]*(x[1])**2+x[0]-10*x[1]+8

func = [func1, func2]


x = np.array([0, 0], dtype=float)
x_0 = x.copy()

grad_func0 = grad(func1)
grad_func1 = grad(func2)

jacobi = [grad_func0(x), grad_func1(x)]

x_1 = x_0 - np.matmul(np.linalg.inv(jacobi),[func1(x_0), func2(x_0)])

jacobi

[array([-10.,   0.]), array([  1., -10.])]

In [ ]:
iteration = 0
maxiteration = 10**2
threshold = 10**(-2)
error = 10**3
t_0 = np.array([0,0], dtype=float)

while error > threshold and (iteration < maxiteration):
    iteration += 1
    t_1 = t_0 - np.matmul(np.linalg.inv(jacobi),[func1(t_0), func2(t_0)])
    error = np.linalg.norm(t_1-t_0)
    t_0 = t_1
    print("x[", iteration, ']=', t_0 )

x[ 1 ]= [0.8  0.88]
x[ 2 ]= [0.94144  0.956096]
x[ 3 ]= [0.98004288 0.98406316]
x[ 4 ]= [0.99288644 0.99419407]
x[ 5 ]= [0.99742453 0.99788152]


In [ ]:
import numpy as np
from autograd import grad

class NewtonRaphsonSolver:
    def __init__(self, equations, variables):
        self.equations = equations
        self.variables = variables
        self.jacobi = self.compute_jacobi()

    def compute_jacobi(self):
        gradient_funcs = [grad(eq) for eq in self.equations]
        jacobi_matrix = np.array([[grad_func(x) for grad_func in gradient_funcs] for x in self.variables])
        return jacobi_matrix

    def solve(self, initial_guess, max_iterations=100, threshold=1e-6):
        iteration = 0
        error = np.inf
        current_values = initial_guess.copy()

        while error > threshold and iteration < max_iterations:
            iteration += 1
            previous_values = current_values.copy()

            try:
                inv_jacobi = np.linalg.inv(self.jacobi(current_values))
                values_evaluated = np.array([eq(current_values) for eq in self.equations])
                update = np.matmul(inv_jacobi, values_evaluated)
                current_values -= update
            except np.linalg.LinAlgError:
                print("Inverse of the Jacobian is not computable.")
                break

            error = np.linalg.norm(current_values - previous_values)

            print("Iteration {}: {}".format(iteration, current_values))

        return current_values

# Example usage:
if __name__ == "__main__":
    # Define your equations and initial guesses
    equations = [
        lambda x: x[0] ** 2 - 10 * x[0] + x[1] ** 2 + 8,
        lambda x: x[0] * x[1] ** 2 + x[0] - 10 * x[1] + 8
    ]

    initial_guess = np.array([0, 0], dtype=float)

    # Create an instance of the solver
    solver = NewtonRaphsonSolver(equations, initial_guess)

    # Solve the system
    solution = solver.solve(initial_guess)
    print("Solution:", solution)

IndexError: ignored